In [126]:
"""
to be deleted when everything is done, I'm just using it to the check the data types and the nan values in all csv files
+ print the markdown table + some other checks
"""
import json
from pathlib import Path

import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

PATHS = [
    # 'stimuli/aoi_texts/aoi/b0.ias',
    'stimuli/word_features/word_features_b0.csv',
    'participants/participant_data.csv',
    'eyetracking_data/fixations/reader0_b0_fixations.csv',
    'eyetracking_data/reader_rm_wf/reader0_b0_merged.csv',
    'eyetracking_data/reading_measures/reader0_b0_rm.csv',
    'eyetracking_data/scanpaths/reader0_b0_scanpath.csv',
    'eyetracking_data/scanpaths_reader_rm_wf/reader0_b0_merged_sp_rm.csv',
]

PATHS_FOLDERS = [
    'stimuli/aoi_texts/aoi/',
    'stimuli/word_features/',
    'eyetracking_data/fixations/',
    'eyetracking_data/reader_rm_wf/',
    'eyetracking_data/reading_measures/',
    'eyetracking_data/scanpaths/',
    'eyetracking_data/scanpaths_reader_rm_wf/',
    'stimuli/stimuli/items.tsv',
    'stimuli/stimuli/stimuli.tsv',
    'participants/',      
]

floats = ['type_length_chars', 
          'lemma_length_chars', 
                     'type_length_syllables', 
    'annotated_type_frequency_normalized', 'type_frequency_normalized',
                     'lemma_frequency_normalized', 'familiarity_normalized', 'regularity_normalized',
                     'document_frequency_normalized', 'sentence_frequency_normalized',
                     'cumulative_syllable_corpus_frequency_normalized',
                     'cumulative_syllable_lexicon_frequency_normalized',
                     'cumulative_character_corpus_frequency_normalized',
                     'cumulative_character_lexicon_frequency_normalized',
                     'cumulative_character_bigram_corpus_frequency_normalized',
                     'cumulative_character_bigram_lexicon_frequency_normalized',
                     'cumulative_character_trigram_corpus_frequency_normalized',
                     'cumulative_character_trigram_lexicon_frequency_normalized',
                     'initial_letter_frequency_normalized',
                     'initial_bigram_frequency_normalized', 'initial_trigram_frequency_normalized',
                     'avg_cond_prob_in_bigrams',
                     'avg_cond_prob_in_trigrams', 'neighbors_coltheart_higher_freq_cum_freq_normalized',
                     'neighbors_coltheart_higher_freq_count_normalized',
                     'neighbors_coltheart_all_cum_freq_normalized',
                     'neighbors_coltheart_all_count_normalized',
                     'neighbors_levenshtein_higher_freq_cum_freq_normalized',
                     'neighbors_levenshtein_higher_freq_count_normalized',
                     'neighbors_levenshtein_all_cum_freq_normalized',
                     'neighbors_levenshtein_all_count_normalized']

In [ ]:
## investigate col values and distribution

values = {}
for folder in PATHS_FOLDERS:
    values[folder] = {}

    first = True
    suffix = '.ias' if folder == 'stimuli/aoi_texts/aoi/' else '.tsv'
    for path in tqdm(Path(folder).glob(f'*{suffix}'), desc=f'Checking files in {folder}'):

        csv = pd.read_csv(path, sep='\t', keep_default_na=False,
                          na_values=['#N/A', '#N/A N/A', '#NA', '-1.#IND', '-1.#QNAN', '-NaN', '-nan', '1.#IND',
                                     '1.#QNAN', '<NA>', 'N/A', 'NA', 'NaN', 'None', 'n/a', 'nan', ''])
        if first:
            all_files = csv
            first = False
        else:
            all_files = pd.concat([all_files, csv], ignore_index=True)

    print(len(all_files))
    cols = all_files.columns.tolist()
    for col in all_files.columns:
        try:
            values[folder][col] += all_files[col].tolist()
        except KeyError:
            values[folder][col] = all_files[col].tolist()

        if all_files[col].dtype in ['float64'] or col in floats:
            print(col)
            cols.remove(col)
            try:
                print(f'min: {all_files[col].min()} max: {all_files[col].max()} mean: {all_files[col].mean()} std: {all_files[col].std()}')
                all_files[col].astype('float64').plot.kde()
                plt.show()

            except ValueError:
                # print value if value if not numeric
                print([v for v in all_files[col].values if not isinstance(v, float)])
        
        elif all_files[col].dtype in ['int64']:
            print(col)
            cols.remove(col)            
            counts = all_files[col].value_counts()
            print(counts.to_dict())
            all_files[col].plot.hist()
            plt.show()

    print(cols)
    print(all_files.dtypes)
    print(all_files['type_length_chars'].value_counts())

In [ ]:
## check duplicate columns
    
all_paths = [
    'stimuli/word_features',
]
cols1 = [
    'PoS_tag'

]
cols2 = [
    'STTS_PoS_tag'
]
    
for path, col1, col2 in zip(all_paths, cols1, cols2):

    files = Path(path).glob('*.csv')

    with open(f'stuff_to_check/mismatch_{Path(path).stem}_{col1}_{col2}.txt', 'w', encoding='utf8') as f:

        for path in files:

            csv = pd.read_csv(path, sep='\t', na_filter=False)

            c1, c2, word = csv[col1], csv[col2], csv['word']

            for idx, (v1, v2, w) in enumerate(zip(c1, c2, word)):
                if v1.lower() != v2.lower():
                    f.write(f'{path}\n')
                    f.write(f'Line: {idx + 1}\n')
                    f.write(f'{col1}: {v1}\t{col2}: {v2}\tword:{w}\n\n')